In [446]:
# -*- coding: utf-8 -*-
"""
Created on Wed Sep  5 09:46:49 2018
@author: Lucas de Moura Rodrigues
@ra: 14.00556-5
"""

'\nCreated on Wed Sep  5 09:46:49 2018\n@author: Lucas de Moura Rodrigues\n@ra: 14.00556-5\n'

In [447]:
import time
import numpy as np
import pandas as pd
from sklearn import linear_model,metrics,svm, preprocessing, ensemble
import sklearn
import matplotlib.pyplot as plt

In [448]:
tempo_inicial = time.time()

In [449]:
# le o csv
print('Le e trata os dados: ', round(time.time() - tempo_inicial, 3))
precos_casa = pd.read_csv("precos_casa_california.csv")

Le e trata os dados:  0.016


In [450]:
# ve as informacoes do Dataframe
precos_casa.describe()

,latitude,median_income
count,20445.000000,20444.000000
mean,35.643596,3.876869
std,2.129236,1.904054
min,32.540000,0.499900
25%,33.940000,2.565750
50%,34.260000,3.540300
75%,37.720000,4.756600
max,41.950000,15.000100


In [451]:
# trata o dataframe, tirando outliers
precos_casa = precos_casa.replace(' ', np.nan)
precos_casa = precos_casa.dropna()
colunas = ['housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income', 'median_house_value']
for i in colunas:
    precos_casa[i] = pd.to_numeric(precos_casa[i])
    precos_casa = precos_casa.loc[precos_casa[i] < precos_casa[i].quantile(0.95)]
precos_casa = precos_casa.reset_index(drop=True)
precos_casa.describe()

,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,13930.000000,13930.000000,13930.000000,13930.000000,13930.000000,13930.000000,13930.000000,13930.000000
mean,35.694599,28.550323,1955.799282,400.786935,1097.638335,374.421608,3.515953,173585.886360
std,2.181956,11.149611,918.063132,174.487808,492.718730,160.755241,1.377513,80856.189371
min,32.540000,1.000000,2.000000,2.000000,3.000000,2.000000,0.499900,0.000000
25%,33.930000,19.000000,1307.250000,275.000000,744.000000,261.000000,2.453700,108000.000000
50%,34.300000,30.000000,1851.000000,387.000000,1057.000000,364.000000,3.360700,162500.000000
75%,37.720000,36.000000,2529.000000,520.000000,1431.000000,488.000000,4.440525,226500.000000
max,41.950000,51.000000,5989.000000,994.000000,2400.000000,735.000000,7.358800,387500.000000


In [452]:
# separa o dataframe
dados = precos_casa.loc[:, precos_casa.columns != 'median_house_value']
target = precos_casa.loc[:, precos_casa.columns == 'median_house_value']

In [453]:
# numeriza o ocean
dic_ocean = {0: 'NEAR BAY', 1: '<1H OCEAN', 2: 'INLAND', 3: 'NEAR OCEAN', 4: 'ISLAND'}
for chave, valor in dic_ocean.items():
    dados = dados.replace(valor, chave)

In [454]:
dados.head(3)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
0,-122.26,37.84,42.0,2555.0,665.0,1206.0,595.0,2.0804,0
1,-122.26,37.85,50.0,1120.0,283.0,697.0,264.0,2.1250,0
2,-122.26,37.84,50.0,2239.0,455.0,990.0,419.0,1.9911,0


In [455]:
# normaliza dados
colunas = dados.columns
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(dados.values.astype(float))
dados = pd.DataFrame(x_scaled)
dados.columns = colunas

#normaliza target
colunas = target.columns
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(dados.values.astype(float))
dados = pd.DataFrame(x_scaled)
dados.columns = colunas

In [456]:
dados.head(3)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
0,0.207951,0.563231,0.82,0.426424,0.668347,0.501877,0.809004,0.230431,0.0
1,0.207951,0.564293,0.98,0.186738,0.283266,0.289529,0.357435,0.236933,0.0
2,0.207951,0.563231,0.98,0.373643,0.456653,0.411765,0.568895,0.217411,0.0


In [457]:
target.head(3)

,median_house_value
0,226700.0
1,140000.0
2,158700.0


In [458]:
# separa o dataframe: dados_treino = 0 - 60%  dados_validacao = 60 - 100%
tamanho_precos_casa = len(precos_casa)
index_60 = int(tamanho_precos_casa * 0.6)  # primeiros 60% do dataframe
dados_treino = dados[:index_60]
dados_validacao = dados[index_60:]
target_treino = target[:index_60]
target_validacao = target[index_60:]
dados_treino = dados_treino.sort_values(by=['median_income','households']).reset_index(drop=True)
dados_validacao = dados_validacao.sort_values(by=['median_income','households']).reset_index(drop=True)
target_validacao = target_validacao.sort_values(by='median_house_value').reset_index(drop=True)
target_treino = target_treino.sort_values(by='median_house_value').reset_index(drop=True)

In [459]:
dados_treino.head(3)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
0,0.638124,0.144527,0.7,0.002171,0.003024,0.005006,0.005457,0.0,0.25
1,0.204893,0.560043,0.9,0.001670,0.002016,0.006258,0.006821,0.0,0.00
2,0.143731,0.731137,0.3,0.068315,0.112903,0.009595,0.023192,0.0,0.50


In [460]:
dados_validacao.head(3)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
0,0.418960,0.587673,0.84,0.016703,0.017137,0.017105,0.015007,0.0,0.5
1,0.754332,0.228480,0.18,0.055955,0.100806,0.043805,0.065484,0.0,0.5
2,0.099898,0.838470,0.28,0.110072,0.145161,0.053400,0.075034,0.0,0.5


In [461]:
target_treino.head()

,median_house_value
0,0.0
1,0.0
2,14999.0
3,14999.0
4,17500.0


In [462]:
target_validacao.head()

,median_house_value
0,14999.0
1,27500.0
2,32500.0
3,32500.0
4,34200.0


In [463]:
# machine learning
print('Predicao dos Dados: ', round(time.time() - tempo_inicial, 3))
#clf = linear_model.LassoLars(alpha=.1)  # 25480756266.08841
#clf = linear_model.Ridge (alpha = .5) # 25490381018.173256
#clf = svm.SVR() # 6922054605.659882
clf = ensemble.BaggingRegressor()
#clf = linear_model.SGDRegressor() # infinito
clf.fit(dados_treino.values, target_treino.values.ravel())
target_predicao = clf.predict(dados_validacao)

Predicao dos Dados:  1.062


In [464]:
target_validacao = target_validacao.sort_values(by=['median_house_value']).reset_index(drop=True)
target_validacao.head()

,median_house_value
0,14999.0
1,27500.0
2,32500.0
3,32500.0
4,34200.0


In [465]:
target_predicao = [float(i) for i in target_predicao]
target_predicao = pd.DataFrame(data=target_predicao,columns=['median_house_value']).sort_values(by=['median_house_value'],ascending=False).reset_index(drop=True)
target_predicao.head()

,median_house_value
0,387320.0
1,387260.0
2,386730.0
3,386710.0
4,386370.0


In [466]:
sklearn.metrics.mean_squared_error(target_validacao, target_predicao)

25436166178.927902

In [467]:
clf.score(dados_validacao, target_validacao)

0.9852447010584179

In [468]:
clf.score(dados_treino, target_treino)

0.9999987579153536

In [469]:
# compara os dados
print('Comparando os Dados: ', round(time.time() - tempo_inicial, 3))
resultado = target_validacao.reset_index(drop=True).copy()
resultado['predicao'] = target_predicao['median_house_value']
resultado = resultado.rename(index=str, columns={'median_house_value': 'Valor Correto', 'predicao': 'Valor_Calculado'})
resultado['Diferenca_Absoluta'] = resultado['Valor Correto'] - resultado['Valor_Calculado']
resultado['Diferenca_Porcentagem'] = (resultado['Valor Correto'] / resultado['Valor_Calculado'] - 1) * 100
resultado.head()

Comparando os Dados:  1.635


,Valor Correto,Valor_Calculado,Diferenca_Absoluta,Diferenca_Porcentagem
0,14999.0,387320.0,-372321.0,-96.127491
1,27500.0,387260.0,-359760.0,-92.898828
2,32500.0,386730.0,-354230.0,-91.596204
3,32500.0,386710.0,-354210.0,-91.595769
4,34200.0,386370.0,-352170.0,-91.148381


In [470]:
# salva o resultado em um excel
excel = pd.ExcelWriter(F"Resultado.xlsx")
precos_casa.to_excel(excel, sheet_name='Precos_Casa_Tratado')
resultado.to_excel(excel, sheet_name='Comparacao')
excel.save()

In [471]:
print('Tempo do Programa: ', round(time.time() - tempo_inicial, 3))

Tempo do Programa:  5.406
